In [1]:
MODEL = "gemma3n:e4b"

# User Query Input

In [4]:
import requests
import json
from IPython.display import display, Markdown, clear_output

def answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None):
    payload = {
        "prompt": prompt,
        "model": model,
        "temperature": temperature,
        "max_new_tokens": 50, # only when stream = False work
        "format": format
    }
    headers = {
        'Content-Type': 'application/json'
    }
    endpoint = "http://localhost:11434/api/generate"

    # Send the POST request with streaming enabled
    with requests.post(endpoint, headers=headers, json=payload, stream=True) as response:
        if response.status_code == 200:
            try:
                # Process the response incrementally
                full_response = ""
                for line in response.iter_lines(decode_unicode=True):
                    if line.strip():  # Skip empty lines
                        response_json = json.loads(line)
                        chunk = response_json.get("response", "")
                        full_response += chunk
                        
                        # Render the response as Markdown
                        if stream:
                            clear_output(wait=True)
                            display(Markdown(full_response))
                        
                return full_response
            except json.JSONDecodeError as e:
                return "Failed to parse JSON: " + str(e)
        else:
            return "Failed to retrieve response: " + str(response.status_code)
        
def multiple_answer_this_prompt(prompt, stream=False, model=MODEL, temperature=0, format=None, n_answers=1):
    answers = []
    for _ in range(n_answers):
        answer = answer_this_prompt(prompt, stream=stream, model=model, temperature=temperature, format=format)
        answers.append(answer)
    return answers

In [ ]:
# Example usage
ans = answer_this_prompt("What is the Big Bang theory?", stream=True)
print("------------------------")
print(type(ans))

In [ ]:
bn_path = "/nets/collection/"


In [5]:
from pydantic import BaseModel

class Query(BaseModel):
  fromNode: str
  toNode: str

prompt = """Is Lung connected to Cancer?"""
ans = answer_this_prompt(prompt, format=Query.model_json_schema())
query = Query.model_validate_json(ans)
print(query)

fromNode='Lung' toNode='Cancer (Lung Cancer)'
